Importing libraries

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

Get Info Box

In [146]:
def get_it(url,title):
    final_url = "https://en.wikipedia.org"+url   # take url for respective film
    r = requests.get(final_url)
    soup = bs(r.content)

    table = soup.find("table",attrs={"class":"infobox vevent"})  # get the table for that mov
    try:    # to catch one extreme edge case of duplicate tag
        headers = table.find_all("th",attrs = {"scope" : "row"})    # get the titles for rows
    except:
        print(soup.prettify())
    d = {}   # to store data
    d['Title'] = title
    rows = table.find_all("tr")    # get the rows of the info_table for a mov
    for row in rows:     # for every row get the values for the respective titles
        header = row.find("th",attrs = {"scope" : "row"})
        if header:
            key = header.get_text(" ").strip() # " " is the string to be used to join the bits of text together
            value = row.find("td").get_text().strip().replace("\xa0"," ").split('\n')
            d[key] = value
    return d


Get the info box for all movies

In [147]:
url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"
r = requests.get(url)
soup = bs(r.content)
# to get all the tables of wiki page, [:-1] except the lst table which contains a few upcoming movies
# and obviously some important data will be missing there.
tables = soup.find_all("table",attrs = {"class" : "wikitable sortable"})[:-1]
lst = []  # to store info_table of every mov
for table in tables:  #for every mov
    films = table.find_all("i")   # get tag of all movies from table and store in films
    for i in films:   # for each film tag
        if i.a:  # if film/mov link exists
            title = i.a.get_text(strip=True) # get title
            link = i.a["href"] # get the film/mov link
#             print(link)  # to debug
            if link=="/wiki/True-Life_Adventures":   # for one extreme edge case where the tags were exactly same for 2 different links    
#                 print("Invalid,passing onto next...") # to debug
                continue
            d = get_it(link,title)   # get data for that mov using the link stored earlier
#             print(link) # to debug
#             print('*'*100) # to debug
            lst.append(d)  # store data of every mov in lst
        else:
            link = "No Link"  # if no link exists for that mov, another edge case and i don't store it
            



In [154]:
len(lst)

437

Saving Our Data

In [156]:
# import json
# with open("wiki_movs.json","w",encoding="utf-8") as f:
#     json.dump(lst,f,ensure_ascii=False,indent=6)

Load Data custom function

In [157]:
def load_json(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

#### Attach Scores With Api


In [222]:
#del lst
mov_info = load_json("wiki_movs.json")

In [223]:
import urllib
def get_scores(title):
    base_url = "http://www.omdbapi.com/?"
    parameters = {"apikey":"70b3cc9f","t":title}
    end_url = urllib.parse.urlencode(parameters)
    final_url = base_url + end_url
    return requests.get(final_url).json()

def get_rts(temp):
    r = temp.get('Ratings',[])
    for ele in r:
        if ele['Source'] == 'Rotten Tomatoes':
            return ele['Value']
    return None

In [224]:
for mov in mov_info:
    title = mov['Title']
    temp = get_scores(title)
    mov['Metascore'] = temp.get('Metascore',None)
    mov['imdbRating'] = temp.get('imdbRating',None)
    mov['Rotten_Tomatoes'] = get_rts(temp)
    mov['Genre'] = temp.get('Genre',None)
    mov['Rated'] = temp.get('Rated',None)
    mov['BoxOffice_Api'] = temp.get('BoxOffice',None)
    mov['Type'] = temp.get('Type',None)


Save Final Data

In [228]:

# Saving to Json

# import json
# with open("Final_data.json","w",encoding="utf-8") as f:
#     json.dump(mov_info,f,ensure_ascii=False,indent=6)

In [231]:
# Saving to CSV

# df = pd.DataFrame(mov_info)
# df.to_csv('Final_Data.csv',index=False)